In [ ]:
from google import genai
from google.genai import types
import gradio as gr
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import io
import json

# --- CONFIGURATION ---
# 1. Get your API key from: https://aistudio.google.com/
# 2. Paste it below inside the quotes
GOOGLE_API_KEY = "MY_APP_KEY"

# Initialize the new Client
client = genai.Client(api_key=GOOGLE_API_KEY)

# --- THE PROCESSOR FUNCTION ---
def extract_and_render_menu(input_image):
    if input_image is None:
        return None

    prompt = """
    Analyze this restaurant menu image.
    Extract the items and prices into a standardized list.
    If item is not clear, extract information from section title, example: Ice creams section with flavor items.
    Return ONLY a raw JSON list of objects with keys 'Product' and 'Price'.
    Do not use Markdown formatting (like ```json). Just the raw JSON.
    Example: [{"Product": "Burger", "Price": "R$20"}]
    """

    try:
        # --- NEW API CALL FORMAT ---
        # The new SDK handles PIL images directly in the 'contents' list
        response = client.models.generate_content(
            model='gemini-2.0-flash', # Or 'gemini-1.5-flash'
            contents=[prompt, input_image]
        )

        text_response = response.text.strip()

        # Clean up if the model accidentally added markdown blocks
        if text_response.startswith("```"):
            text_response = text_response.strip("`").replace("json\n", "").replace("JSON\n", "")

        data = json.loads(text_response)

    except Exception as e:
        return create_error_image(f"Error: {str(e)}")

    # 2. Convert to DataFrame
    df = pd.DataFrame(data)

    # 3. Render DataFrame as an Image
    return dataframe_to_image(df)



# --- HELPER: CONVERT DATAFRAME TO IMAGE ---
def dataframe_to_image(df):
    """
    Renders a Pandas DataFrame as a matplotlib plot and returns a PIL Image.
    """
    fig, ax = plt.subplots(figsize=(8, len(df) * 0.5 + 1)) # Dynamic height
    ax.axis('tight')
    ax.axis('off')

    # Create the table
    the_table = ax.table(cellText=df.values,
                         colLabels=df.columns,
                         cellLoc = 'center',
                         loc='center')

    # Styling the table
    the_table.auto_set_font_size(False)
    the_table.set_fontsize(12)
    the_table.scale(1.2, 1.2)

    # Convert plot to PIL Image
    buf = io.BytesIO()
    plt.savefig(buf, format='png', bbox_inches='tight', dpi=150)
    plt.close(fig)
    buf.seek(0)
    return Image.open(buf)

def create_error_image(text):
    """Creates a simple image with error text."""
    fig, ax = plt.subplots()
    ax.text(0.5, 0.5, text, ha='center', va='center', wrap=True)
    ax.axis('off')
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    plt.close(fig)
    buf.seek(0)
    return Image.open(buf)


# --- GRADIO INTERFACE ---
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# 🍽️ AI Menu Digitizer")
    gr.Markdown("Upload a photo of a menu. Gemini will extract the items and generate a clean table image.")

    with gr.Row():
        with gr.Column():
            input_img = gr.Image(type="pil", label="Input Menu Image", height=400)
            with gr.Row():
                clear_btn = gr.ClearButton(components=[input_img])
                submit_btn = gr.Button("Submit", variant="primary")

        with gr.Column():
            output_img = gr.Image(label="Standardized Table Output", height=400)

    # Wire up the buttons
    submit_btn.click(
        fn=extract_and_render_menu,
        inputs=input_img,
        outputs=output_img
    )

# Launch the app
demo.launch(debug=True)